In [13]:
import pandas as pd
import os
import numpy as np
from PIL import  Image
import matplotlib.pyplot as plt

folder_path = '../Emulation'


# Report on Uncertainty Quantification and Calibration of the Cardiac Model ModularCirc

## Introduction

This report details the work conducted for WP3, the uncertainty quantification and calibration of cardiac models. The primary model used for this study was the 0D model given in Korakianitis and Shi (Figure 1), from which we simulated right heart pressure traces and cardiac output. We focused on calibrating five key model parameters while keeping the others fixed. These five parameters were selected via a gloal sensitivity analysis of the model and through expert advice from other members of the project. All simulations were performed using Max's `ModularCirc`. 

![Figure 1: Korakianitis and Shi Model](KorakianitisModel_circuit.png)

*Figure 1: Korakianitis and Shi Model.*


## Methodology

### Sampling and Simulation

Synthetic data was created using the 0D Korakianitis and Shi simulator.

- **Model Parameters:** 

 To simplify our methodology, only five of the model parameters were sampled namely $R_{pat}, C_{pat}, C_{svn}$, the right-ventricle activation function, $rv.E_{act}$, and duration of pulse, $T$. All other parameters were fixed to the point estimates given in the Korakianitis paper. The parameters were sampled 100 times using the Sobol method. The point estimate of each sampled parameter along with their range given by their scaling factors are given in Table 1. 

| Parameter    | Point Estimate| Scaling Factor (Min) | Scaling Factor (Max) |
|:-------------|:------:|:-------:|:-------:|
| $R_{pat}$    | 0.31   | 0.5     | 1.5     |
| $C_{pat}$    | 3.8    | 0.5     | 1.5     |
| $C_{svn}$    | 20.5   | 0.5     | 1.5     |   
| $rv.E{act}$  | 1.15   | 0.5     | 3       |
| $T$          | 1      | 0.4     | 0.96    |

*Table 1: Point estimates of parameters with sampling range given by the minimum and maximum scaling factor value.* 


- **Simulation Model:** 

We used ModularCirc to run simulations of Korakianitis and Shi's model formulation of the cardiac loop. From this we recorded pressure transients in the right arterial tree and cardiac output. Figure 2 shows a plot of the 100 simulated pressure transients. 

![Figure 2: Pressure Traces](pressure_traces.png)

*Figure 2: Plot of 100 Simulated Pressure Transients in the Arterial Tree.*





### Principal Component Analysis (PCA)

- **PCA of Pressure Trace:** 

A principle component analysis was conducted on the pressure traces. Figure 3 shows histograms of the first 10 principle compoments of the pressure traces. Figure 4 shows the variance explained and cumulative variance of the principle components. We found that over $99\%$ of the variance could be explained using the first three principle components.


![Figure 3: PCA Pressure Traces](PCA_histogram.png)

*Figure 3: Histograms of the first 10 principle components of the pressure traces.*

![Figure 4: PCA Pressure Traces](PCA_variance_explained.png)

*Figure 4: Left: Variance explained by each principle components of the pressure trace. Right: Cumulative sum of variance explained by each principle component of the pressure trace.*

- **Principal Components Recorded:** 

As over $99\%$ of the variance could be explained using the first three principle components, we chose to build an emulator for these rather than the whole pressure trace. 



## Emulator Development

We developed emulators for outputs of the model that we believe are observable in patients and of interest in light of pulmonary arterial hypertension. We developed a single emulator for each of the first three principle components of the pressure trace (PC1, PC2 and PC3), cardiac output (CO) and summary statistics of the pressure traces namely the maximum, minimum and mean pressure. Additionally, we developed emulators of the first three principle components of the pressure trace plus CO, and also the principle components of the pressure trace, CO and the duration of the pulse. We intially developed Gaussian process emulators for each output. 

### Gaussian Process Emulators

Each emulator used a radial basis function kernel and zero mean function.  We trained the emulator on $80\%$ of the simulated data and reserved the other $20\%$ for testing the emulator's predictive power. 

**GP Emulator Predicitve Power, $R^2$**

In each case, $R^2$ was calculated for the error between the simulated output (e.g. PC1) and the predicted value. For all emulators, the $R^2$ between the simulated and predicted values was $>0.98$. The trained emulators and their respective $R^2$ are gieven below. 



In [14]:
GP_emulator_results = pd.read_pickle(f'{folder_path}/Outputs/Emulators/RBF_models_and_r2_scores.csv')
GP_emulator_results

,R2_Score,Model
y_PC1,0.998637,\nName : GP regression\nObjective : 82.9228280...
all_y_PC1,0.998650,\nName : GP regression\nObjective : 82.6900306...
CO_y_PC1,0.998637,\nName : GP regression\nObjective : 82.9228280...
y_PC2,0.997759,\nName : GP regression\nObjective : -8.5241813...
all_y_PC2,0.997584,\nName : GP regression\nObjective : -9.3995655...
CO_y_PC2,0.997759,\nName : GP regression\nObjective : -8.5241813...
y_PC3,0.984687,\nName : GP regression\nObjective : -19.126323...
all_y_PC3,0.991417,\nName : GP regression\nObjective : -30.904616...
CO_y_PC3,0.984686,\nName : GP regression\nObjective : -19.126323...
CO,0.986648,\nName : GP regression\nObjective : -126.19235...


NB: y_PC* are the emulators of the principle components of the pressure traces only. all_y_PC* are the emulators of the principle components of the pressure traces, cardiac output and duration of pulse. CO_y_PC* are the emulators of the principle components of the pressure traces and cardiac output. 


### Linear Regression Models

As the $R^2$ for the Gaussian process emulators was close to 1 in all cases, it seems natural to investigate whether a linear regression emulator would fit just as well and thus save considerable computing power. We fit a seperate linear regression model to each of the outputs PC1, PC2, PC3, CO, maximum pressure, minimum pressure, and mean pressure.

**Linear Emulator Predicitve Power, $R^2$**

Again, in each case $R^2$ was calculated for the error between the simulated output (e.g. PC1) and the predicted value. All linear regression models fitted well to the data and had relatively good $R^2$ values ($>0.8$). However, the emulator for PC3 did not perform as well and had a $R^2$ value of 0.65 (to 2d.p.). The trained emulators and their respective $R^2$ are gieven below. 


In [15]:
linear_emulator_results = pd.read_pickle(f'{folder_path}/Outputs/Emulators/linear_models_and_r2_scores.csv')
linear_emulator_results

,R2_Score,Model
y_PC1,0.987922,LinearRegression()
all_y_PC1,0.987971,LinearRegression()
CO_y_PC1,0.987922,LinearRegression()
y_PC2,0.882277,LinearRegression()
all_y_PC2,0.888562,LinearRegression()
CO_y_PC2,0.882277,LinearRegression()
y_PC3,0.647736,LinearRegression()
all_y_PC3,0.873160,LinearRegression()
CO_y_PC3,0.647736,LinearRegression()
CO,0.940900,LinearRegression()


NB: y_PC* are the emulators of the principle components of the pressure traces only. all_y_PC* are the emulators of the principle components of the pressure traces, cardiac output and duration of pulse. CO_y_PC* are the emulators of the principle components of the pressure traces and cardiac output. 


## Calibration of Linear Regession Model

As these models are linear, calibration is straightforward and just requires computing the inverse problem via linear algebra. In mathematical notation, we have $$y = \beta X + c + \epsilon$$ where $y$ is our output (e.g. PC1, maximum pressure etc), X is the matrix of sampled input parameters and $\beta$ is the vector of model coefficients. 

Then, for any given observation $y_{obs}$, the inverse problem just requires computing $$x^*=(y_{obs}-\beta_0)(\beta^T \beta)^{-1} \beta^T$$
where $\beta_0$ is the intercept of the linear model. 

Firstly we calibrate each model individually, computing a mean squared error statistic to assess performance. Then we extend our methodolgy to calibrate all models simultaneously.

### Single Output Calibration 

As discussed, we first calibrate each model individually. It is worth noting here that since we have five input parameters to calibrate and one observation, $y_{obs}$, we can only find the minimum norm solution. This is unlikely to be close to the actual values of the input parameters which led to the observation. Thus, once calibrated we feed the estimated input parameters back into the linear model to assess whether we can recover the observation value. In theory we only need to calibrate each model using a single observation, however, for brevity I conduct multiple independent calibrations on each model using a range of 20 observations and calculate MSE on the difference between $y^*$, the output predicted using the calibrated inputs and the true value of the observation, $y_{obs}$. 

- **MSE of $y_{obs}$ and $y$ for all linear models:** 

Below is a table of the MSEs between the output of the calibrated model  and the (true) simulated output for each linear model. 

In [16]:
linear_calibration_results = pd.read_csv(f'../LinearCalibration/linear_model_calibration_mse_results.csv')
calibrated_max_press = pd.read_csv(f'../LinearCalibration/calibrated_max_press.csv')

linear_calibration_results


,Unnamed: 0,MSE
0,model_y_PC1,6.557406e-30
1,model_all_y_PC1,3.540260e-29
2,model_CO_y_PC1,6.557406e-30
3,model_y_PC2,1.117425e-31
4,model_all_y_PC2,5.669938e-32
5,model_CO_y_PC2,1.117425e-31
6,model_y_PC3,8.017652e-31
7,model_all_y_PC3,1.369875e-30
8,model_CO_y_PC3,8.017652e-31
9,model_CO,1.873545e-31


- **Recovered Estimates:** 

As an example here are the values of the 20 simulated outputs and their respective values from calibrated model of maximum pressure.

In [17]:
calibrated_max_press

,y_true,y_calibrated
0,22.551249,22.551249
1,15.931235,15.931235
2,28.465645,28.465645
3,24.604022,24.604022
4,22.291235,22.291235
5,17.320841,17.320841
6,26.449743,26.449743
7,28.691640,28.691640
8,31.187409,31.187409
9,31.023888,31.023888



### Multiple Output Calibration (PC1, PC2, PC3, Cardiac Output, Max Pressure)

The next step is to calibrate multiple models simulataneously. Specifically we calibrated models for PC1, PC2, PC3, CO and maximum pressure together. Here we hope that we are able to recover a better approximation to the true solution of the inverse problem. In mathematical notation, we are solving the following. $$y = \beta X + c + \epsilon$$ where $y$ is the column vector of multiple outputs, $X$ is the vector of input parameters and $\beta$ is the matrix of model coefficients. More specifically, $$y = \begin{bmatrix}
PC1 \\
PC2 \\
PC3 \\
CO  \\
max(press) 
\end{bmatrix} $$

$$ X^T = \begin{bmatrix}
C_{svn} & R_{pat} & C_{pat} & RV.E_{act} & T 
\end{bmatrix} $$

and 
 
$$ \beta = \begin{bmatrix}
\beta_{11} & \beta_{12} & \beta_{13} & \beta_{14} & \beta_{15} \\
\beta_{21} & \beta_{22} & \beta_{23} & \beta_{24} & \beta_{25} \\
\beta_{31} & \beta_{32} & \beta_{33} & \beta_{34} & \beta_{35} \\
\beta_{41} & \beta_{42} & \beta_{43} & \beta_{44} & \beta_{45} \\
\beta_{51} & \beta_{52} & \beta_{53} & \beta_{54} & \beta_{55} \\
\end{bmatrix}$$


### Calibration Results:

We calibrated the model with a single observation. Below is a table of values for the true and calibrated input parameters. Similar to the single output models, the calibrated values were again fed back into the linear model to see if the true outputs can be recovered. 



In [18]:
calibration_folder = '../LinearCalibration'
multi_output_results = pd.read_csv(f'{calibration_folder}/multiple_output_calibration_result_x.csv')
calibrated_multi_output = pd.read_csv(f'{calibration_folder}/multiple_output_calibration_result_y.csv')
multi_output_results.columns.values[0] = 'Parameter'
multi_output_results


,Parameter,x_true,x_calibrated
0,# svn.c,24.992088,-16.789781
1,pat.r,0.155291,0.138597
2,pat.c,5.101243,7.688300
3,rv.E_act,3.333386,1.310899
4,T,0.614289,1.124573


In [19]:
calibrated_multi_output

,y_true,y_calibrated
0,-17.616349,-17.616349
1,-1.624573,-1.624573
2,-0.841759,-0.841759
3,3.877907,3.877907
4,14.946666,14.946666


The calibrated input parameters do not closely match the true values. However, $R_{pat}$ appears to be recovered to some extent. To explore this further, we performed 20 independent calibration attempts across a range of observation values and calculated the MSE for each true input parameter against its corresponding calibrated value. The MSE for each calibrated parameter is given in the table below. 

In [20]:
MSE_x = pd.read_csv(f'{calibration_folder}/MSE_multi_output_x.csv')
MSE_x.columns.values[0] = 'Parameter'
MSE_x

,Parameter,MSE
0,# svn.c,4581.203286
1,pat.r,0.000582
2,pat.c,22.615799
3,rv.E_act,4.106932
4,T,0.663989


Here we have used the mean squared error of calibrated parameter and its true value to determine how well each parameter has been calibrated. However, other work e.g. Bjordalsbakke et al have use other cost functions such as those which directly compare the pressure or volume waveforms. 

### Bayesian Calibration  
Assuming we have the following linear regression model $$y = \beta X + \epsilon_d,$$ where $$\epsilon_d \sim N_p(0, \sigma^2I)$$ and $I$ is the $d$-dimensional identity matrix. Here $d=5$, the dimension of $X$. To be explicit, $dim(y) = dim(X) = 5$ and $dim(\beta) = (5,5)$.

Then it follows that we have the distribution $$y|X \sim N(\mu, \Sigma)$$ where $$\mu = \beta X , \Sigma = \sigma^2I.$$

We will put priors on the parameters we are trying to infer. The input parameters $X$ were sampled uniformly from the table below. However, for simplicity, I will put a Gaussian prior on each parameter such that approximately 99% of the data is accounted for within 3 standard deviations of the mean. The priors are as follows.

| Parameter    | Point Estimate| Scaling Factor (Min) | Scaling Factor (Max) | Prior |
|:-------------|:------:|:-------:|:-------:| :------: |
| $C_{svn}$    | 20.5   | 0.5     | 1.5     |   $C_{svn} \sim N(20.5, 3.42)$      |
| $R_{pat}$    | 0.31   | 0.5     | 1.5     |   $R_{pat} \sim N(0.31, 0.05)$      |
| $C_{pat}$    | 3.8    | 0.5     | 1.5     |   $C_{pat} \sim N(0.38, 0.63)$      |
| $rv.E{act}$  | 1.15   | 0.5     | 3       |   $rv.E_{act} \sim N(1.15, 0.48)$   |
| $T$          | 1      | 0.4     | 0.96    |   $T \sim N(1, 0.15)$               |

Thus our prior on $X$ is given by $X \sim MVN(\mu_0, \Sigma_0)$ where,  
$$ \mu_0 = \begin{bmatrix}
20.5 \\
0.31 \\
0.38 \\
1.15  \\
1 
\end{bmatrix} ,


\Sigma_0 = \begin{bmatrix}
3.42 & 0 & 0 & 0 & 0 \\
0 & 0.63 & 0 & 0 & 0  \\
0 & 0 & 0.05 & 0 & 0  \\
0 & 0 & 0 & 0.48 &  0 \\
0 & 0 & 0 & 0 & 0.15 
\end{bmatrix}
$$

The posterior distribution of $x|y$ is given as $$\pi(X|y) = \pi(y|X)\pi(X).$$ With some (omitted) calculations it can be shown that $$\pi(X|y) \sim N(\mu_{post}, \Sigma_{post})$$ where $$\mu_{post} = \Sigma_{post}\left(\frac{\beta^T y}{\sigma^2} + \Sigma_0^{-1}\mu_0\right)$$ and $$ \Sigma_{post} = \left( \frac{\beta^T\beta}{\sigma^2} + \Sigma_0^{-1}\right)^{-1}.$$

We will derive the above posterior means and standard deviation for our data. We assume that measurement error is know to have mean 0 and $\sigma^2 = 1$ for each output. NB: This is a huge assumption and we will investigate heterogenous measurement errors later.

### Bayesian Calibration Results:

We calibrated the model with a single observation made from our 5 outputs (PC1, 2 and 3 and CO, max(pressure)). Below is a table of values for the true and calibrated input parameters. 



In [21]:
bayesian_results = pd.read_csv(f'{calibration_folder}/bayesian_calibration_result_x.csv')
bayesian_results.columns.values[0] = 'Parameter'
bayesian_results


,Parameter,x_true,x_calibrated
0,# svn.c,24.992088,22.738233
1,pat.r,0.155291,0.198214
2,pat.c,5.101243,5.115237
3,rv.E_act,3.333386,1.235067
4,T,0.614289,0.924951


The prior (orange) and updated posterior (blue) distribution are given as diagonal plots in the figure below along with the true parameter value (red dashed). The off-diagnoal plots give pairwise plots of samples of the parameter space drawn form the posterior distribution. 


![Figure 5: Posterior distribtuion of calibrated parameters](calibrate_post_noise1.png)


*Figure 5: Posterior pairwise relationships of calibrated parameters .*




When computing the inverse problem multiple times, we can see form the MSE of the calibrated parameters that the bayesian calibration provides smaller errors that the frequentist approach.

### Frequentist MSE

In [22]:
mse_ = pd.read_csv(f'{calibration_folder}/MSE_multi_output_x.csv')
mse_.columns.values[0] = 'Parameter'
mse_

,Parameter,MSE
0,# svn.c,4581.203286
1,pat.r,0.000582
2,pat.c,22.615799
3,rv.E_act,4.106932
4,T,0.663989


### Bayesian MSE

In [23]:
bayesian_mse_noise1 = pd.read_csv(f'{calibration_folder}/bayes_MSE_multi_output_x.csv')
bayesian_mse_noise1.columns.values[0] = 'Parameter'
bayesian_mse_noise1


,Parameter,MSE
0,# svn.c,44.044008
1,pat.r,0.002097
2,pat.c,0.808101
3,rv.E_act,1.149479
4,T,0.074652


### Heterogeous Observation Errors

We now focus on incorporating more realistic heterogeous observations errors. We generalise our formulation of the posterior mean and covariance matrix to the following. 




$$\mu_{post} = \Sigma_{post}\left(\beta^T \Sigma_\epsilon^{-1} y + \Sigma_0^{-1}\mu_0\right)$$ 
$$\Sigma_{post} = \left(\beta^T \Sigma_\epsilon^{-1} \beta + \Sigma_0^{-1}\right)^{-1}$$



where $\Sigma_\epsilon = diag(\sigma_1, \dots \sigma_q)$ where $q$ is the dimensions of $y$ i.e. the number of outputs.

We initially take $\sigma_i$ to be 5% of the standard deviation of 100 simulated observations for each output $i$, i.e. the observation error is set to 5% for each output. The observation errors are assumed to be independent. Thus our covariance matrix for $\Sigma_\epsilon$ is given as 



In [24]:
combined_output_data = pd.read_csv(f'{calibration_folder}/combined_outptut_data.csv')
obs_error = np.std(combined_output_data)*0.05
epsilon = np.diag(obs_error)
pd.DataFrame(epsilon)

 /opt/anaconda3/envs/ModularCirc/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3643: FutureWarning:The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)


,0,1,2,3,4
0,0.483382,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.110475,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.060462,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.017034,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.250418


The below plot gives


![Figure 6: Posterior distribtuion of calibrated parameters with 5% observation error.](calibrate_post_noise_0.05.png)


*Figure 6: Posterior distribtuion of calibrated parameters with 5% observation error .*



## Discussion

- **Comparison of Emulators:** 

In this work we have compared Guassian process emulators to Linear regression models. We found that the predicitve power of Gaussian process emulators with an RBF kernel had high predicitve power with emulators for each model having an $R^2$ above 0.88. As the $R^2$ were so high for the GP emulators we experimented with fitting linear regression models to our data. We found the $R^2$ of these models to be typically lower but still performing exceptionally well ($R^2 > 0.8$ for all models except models of the third principle components which had an $R^2 > 0.65$).

- **Calibration Accuracy:** 

In the first instance we calibrated our five input parameters to a single output (e.g. maximum pressure). As we have four degress of freddom here, we were only able to approximate the minimum norm solution to the inverse problem of calibration. However, upon feeding the (minimum norm solution) calibrated inputs back into the linear models we were able to recover the true observation values well. The MSE between the true observation value and the value from our model using our calibrated inputs was very close to zero in all models ($ < 0.3 *10^{-29}$). 

Following this we calibrated multiple outputs simultaneously. Specifically we calibrated models for PC1, PC2, PC3, CO and maximum pressure. We found that whilst the calibrated inputs were still not close estimates to the true values on the whole, the calibrated $R_{pat}$ was very close its true value (MSE < 0.0005).  


## Conclusion

Linear regression models may be a quick and convinient alternative to GP emulators for the caridac data. 

We are able to recover observational data via our calibrated models and possibly accurately calibrate $R_{pat}$ to its true value. 

- **Future Work:** 

We should begin calibrating the models in a Bayesian ay using MCMC etc.

